In [5]:
openaikey="x"
import os
os.environ["OPENAI_API_KEY"]=openaikey
os.environ["HUGGINGFACEHUB_API_TOKEN"]="x"
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = "x"

# Twitter Profile

In [38]:
#!pip install ntscraper
from ntscraper import Nitter
import json
import csv
import pandas as pd


username=input("")


tweets = Nitter().get_tweets(f"{username}", mode='user', number=100)
with open(f"{username}.json", "w") as file:
    json.dump(tweets, file, indent=4)


# ---------------------Save data---------------------

with open(f"{username}.json", 'r') as file:
    json_data = json.load(file)
df = pd.json_normalize(json_data['tweets'])

df.to_csv(f'{username}.csv', index=False)
print("CSV file saved successfully.")

KeyboardInterrupt: Interrupted by user

# Setup

In [12]:
!pip install langchain_google_genai
!pip install PyPDF2
!pip install langchain
!pip install faiss-cpu #gpu
!pip install langchain_openai

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [19]:
import warnings
warnings.filterwarnings('ignore')
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import DataFrameLoader
import pandas as pd

df=pd.read_csv(f"{username}.csv")
seelcted_columns=["text","date","quoted-post.text"]
df_new=df[seelcted_columns]

loader = DataFrameLoader(df_new, page_content_column="text")

def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks

chunks=get_text_chunks(str(loader.load()))
vector_store = FAISS.from_texts(chunks, OpenAIEmbeddings())


In [20]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_retrieval_response",
            "description": "Get responses from documents, texts, files, pdf's etc",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_input": {
                        "type": "string",
                        "description": "Query about the documents",
                    },
                    "doc_data":{
                        "type":"string",
                        "description": "data related to the user input",
                    },

                },
                "required": ["user_input","index_number"]
            },
        }
    },

]

In [21]:
def get_retrieval_response(prompt):
  query = prompt
  docs = vector_store.similarity_search(query)
  return docs

In [31]:
from openai import OpenAI
client = OpenAI()
def get_reponse(prompt):
  completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
      {"role": "system", "content": "You are my helpful Twitter LLM (Large Language Model), your name is TLMM"},
      {"role": "user", "content":  f"{prompt}"}
    ],
    tools=tools
  )

  if completion.choices[0].message.content!=None:
    print(completion.choices[0].message.content)
  #print(completion.choices[0].message.tool_calls)
  if completion.choices[0].message.tool_calls!=None:
    #print("Function is running...")
    docs=get_retrieval_response(prompt)
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
      {"role": "system", "content": "You are my helpful Twitter LLM (Large Language Model), your name is TLMM"},
      {"role": "user", "content":  f"Use the following pieces of context to answer the user's question. If you can't find the answer in context, just say that you don't know, don't try to make up an answer. {prompt} in this context: {docs}."}
    ]
  )
    print(completion.choices[0].message.content)



# Run

In [39]:
while True:
  prompt=input("")
  if prompt!="exit":
    query = prompt
    docs = vector_store.similarity_search(query)
    get_reponse(prompt)

  else:
    break

hi who are you
Hello! I am TLMM, your helpful Twitter LLM (Large Language Model). How can I assist you today?
what are the most recent topics in context
The most recent topics in the given context are related to discussions about freedom of speech, censorship, government-industrial complex, disinformation, birth control pill side effects, powerwall technology, and SpaceX achievements.
thoughts on freedom of speech 
In the context provided, there are different viewpoints presented on freedom of speech. One post mentions, "Free speech is the bedrock of democracy. That’s why it’s the FIRST Amendment. Without free speech, all is lost." This indicates a strong support for the importance of freedom of speech in maintaining democracy.

On the other hand, there is a post discussing concerns about government interference in censorship of free speech, highlighting a legal battle where it's claimed that the Biden administration coerced social media companies into suppressing free speech on variou